### **Movie Recommendation System**

In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ratings = pd.read_csv('/content/drive/MyDrive/Foundational_Projects/AI/Recommendation System/ratings.csv')

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,877,4155,5,1651201566
1,305,7661,2,1639553712
2,381,8423,2,1610704432
3,208,6433,1,1650223767
4,47,7752,4,1663998365


In [4]:
ratings['movieId'].nunique()

9742

In [5]:
movies = pd.read_csv('/content/drive/MyDrive/Foundational_Projects/AI/Recommendation System/movies.csv')

In [6]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies['title'].nunique()

9737

In [8]:
movies['genres'].nunique()

951

In [9]:
user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')

In [12]:
# Fill NaNs with 0 — or better, leave them as NaN for cosine similarity
user_movie_matrix_filled = user_movie_matrix.fillna(0)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

user_similarity = cosine_similarity(user_movie_matrix_filled)

user_similarity_df = pd.DataFrame(user_similarity,
                                   index=user_movie_matrix.index,
                                   columns=user_movie_matrix.index)

In [14]:
def get_liked_movies(user_id, ratings, movies, min_rating=4):
    """Return list of movies that the user rated >= min_rating"""
    liked = ratings[(ratings['userId'] == user_id) & (ratings['rating'] >= min_rating)]
    liked = liked.merge(movies, on='movieId')
    return liked[['title', 'rating']].sort_values(by='rating', ascending=False)

In [15]:
def get_user_recommendations(user_id, user_movie_matrix, user_similarity_df, top_n=5):
    # Get similarity scores for this user with all others
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]  # skip self

    # Movies this user already watched
    user_rated_movies = user_movie_matrix.loc[user_id].dropna().index

    # Weighted ratings from similar users
    weighted_scores = pd.Series(dtype=float)

    for sim_user, sim_score in similar_users.items():
        sim_user_ratings = user_movie_matrix.loc[sim_user]
        # Only consider movies the target user hasn't watched
        sim_user_ratings = sim_user_ratings.drop(user_rated_movies, errors='ignore')
        weighted_scores = weighted_scores.add(sim_user_ratings * sim_score, fill_value=0)

    # Normalize by similarity sum
    normalization = similar_users.sum()
    if normalization > 0:
        weighted_scores = weighted_scores / normalization

    # Get top-N recommended movie IDs
    recommended_movie_ids = weighted_scores.sort_values(ascending=False).head(top_n).index

    # Map to titles
    movie_map = dict(zip(movies['movieId'], movies['title']))
    recommendations = [movie_map.get(mid, f"Movie {mid}") for mid in recommended_movie_ids]

    return recommendations

In [16]:
def full_recommendation_flow(user_id, ratings, movies, user_movie_matrix, user_similarity_df, top_n=5):
    print(f"\n🎬 Movies liked by User {user_id} (rated ≥ 4):\n")
    liked_movies = get_liked_movies(user_id, ratings, movies)

    if liked_movies.empty:
        print("No highly rated movies found for this user.")
        return

    for title, rating in liked_movies.values:
        print(f"⭐️ {title} (Rating: {rating})")

    print(f"\n🎯 Based on similar users, we recommend:\n")
    recommendations = get_user_recommendations(user_id, user_movie_matrix, user_similarity_df, top_n)

    for movie in recommendations:
        print("👉", movie)

In [19]:
full_recommendation_flow(1, ratings, movies, user_movie_matrix, user_similarity_df, top_n=5)


🎬 Movies liked by User 1 (rated ≥ 4):

⭐️ Kentucky Fried Movie, The (1977) (Rating: 5)
⭐️ Solaris (2002) (Rating: 5)
⭐️ Something to Talk About (1995) (Rating: 5)
⭐️ Flash Gordon Conquers the Universe (1940) (Rating: 5)
⭐️ Last Exit to Brooklyn (1989) (Rating: 5)
⭐️ City by the Sea (2002) (Rating: 5)
⭐️ This Is Spinal Tap (1984) (Rating: 5)
⭐️ Envy (2004) (Rating: 5)
⭐️ Spaced Invaders (1990) (Rating: 5)
⭐️ African Queen, The (1951) (Rating: 5)
⭐️ Galaxy Quest (1999) (Rating: 4)
⭐️ Dr. Dolittle (1998) (Rating: 4)
⭐️ Confessions of a Dangerous Mind (2002) (Rating: 4)
⭐️ Quest for Camelot (1998) (Rating: 4)
⭐️ My Bloody Valentine (1981) (Rating: 4)
⭐️ Motorcycle Diaries, The (Diarios de motocicleta) (2004) (Rating: 4)
⭐️ Torn Curtain (1966) (Rating: 4)
⭐️ Jurassic Park (1993) (Rating: 4)

🎯 Based on similar users, we recommend:

👉 Movie 8240
👉 Movie 2842
👉 Movie 5332
👉 Bikini Beach (1964)
👉 Movie 6960
